In [1]:
from scipy import ndimage
from skimage import io,util,color
from IPython import display
import numpy
import sys
import math
import random

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

In [2]:
def readAndShowImage(image):
    im = io.imread(image)
    io.imshow(im)
    io.show()

inputImage = widgets.Dropdown(
    options={
        'Strand':'strand.jpeg',
        'Mandrill':'mandrill.png',
        'Trump':'trump.jpg',
        'Gruppe':'gruppe.jpg',
        'RGB':"rgb.png",
        'Dress':'TheDress.png'
        },
    description='Bilde: ',
    disabled=False,
)

interactive(readAndShowImage, image=inputImage)

interactive(children=(Dropdown(description='Bilde: ', options={'Strand': 'strand.jpeg', 'Mandrill': 'mandrill.…

In [11]:
def distance(pixel1, pixel2):
    return ((pixel1[0] - pixel2[0])**2 + (pixel1[1] - pixel2[1])**2 + (pixel1[2] - pixel2[2])**2)**0.5

def getKmeansPalette(im, numSegments, tresh):
    width, height, depth = im.shape
    
    segments = []
    
    #Plukke ut N tilfeldige pixler som skal brukes som start punkter
    for x in range(numSegments):
        segment = {}
        segment["pixel"] = im[random.randint(0,width-1),random.randint(0,height-1)]
        segment["data"] = {"sum":[0,0,0],"num":0}
        segments.append(segment)
    
    while(True):
        #Finne gjennomsnittsfargen for hvert punkt
        for x in range(width):
            for y in range(height):
                closest = 0
                closestDistance = math.inf
                for segment in segments:
                    dist = distance(im[x,y],segment["pixel"])
                    if( dist < closestDistance):
                        closestDistance = dist
                        closest = segment
                closest["data"]["sum"] += im[x,y]
                closest["data"]["num"] += 1

        #Flytter punktet til gjennomsnittetsfargen til de nærmeste pixlene
        movement = 0
        for segment in segments:
            segment["data"]["prevPixel"] = segment["pixel"]
            if(segment["data"]["num"] != 0):
                segment["pixel"] = segment["data"]["sum"] / segment["data"]["num"]
            else:
                segment["pixel"] = [0,0,0]
            movement += distance(segment["data"]["prevPixel"],segment["pixel"])
            segment["data"]["sum"] = [0,0,0]
            segment["data"]["num"] = 0
        
        print(movement/numSegments)
        if (movement/numSegments < tresh):
            break
            
    return [segment["pixel"] for segment in segments]

def getClosestFromPallette(pixel,palette):
    closest = 0
    closestDistance = math.inf
    for segment in palette:
        dist = distance(pixel,segment)
        if( dist < closestDistance):
            closestDistance = dist
            closest = segment
    return closest

def segment(im,palette):
    im = numpy.array(im, copy=True)    
    return numpy.apply_along_axis(getClosestFromPallette,2,im,palette)

In [4]:
def segmentDithering(im, palette):
    width, height, depth = im.shape
    
    ret = numpy.array(im, copy=True)  
    
    for y in range(0,height):
        for x in range(0,width):
            closest = 0
            closestDistance = math.inf
            for segment in palette:
                dist = distance(ret[x,y],segment)
                if( dist < closestDistance):
                    closestDistance = dist
                    closest = segment
                        
            oldpixel  = ret[x,y]
            newpixel  = closest   
            
            quant_error  = oldpixel - newpixel
            
            ret[x,y]  = newpixel
                        
            if(x < width - 1):
                ret[x + 1,y    ] = numpy.clip(ret[x + 1,y    ] + quant_error * (7 / 16),0.0,1.0)
            
            if(y < height - 1):
                ret[x    ,y + 1] = numpy.clip(im[x    ,y + 1] + quant_error * (5 / 16),0.0,1.0)
                
            if(x != 0 and y < height - 1):
                ret[x - 1,y + 1] = numpy.clip(ret[x - 1,y + 1] + quant_error * (3 / 16),0.0,1.0)
            
            if(x < width - 1 and y < height - 1):
                ret[x + 1,y + 1] = numpy.clip(ret[x + 1,y + 1] + quant_error * (1 / 16),0.0,1.0)
            
    
    return ret

In [5]:
def getGrayscalePalette(numSegments):
    segments = []
    for x in range(numSegments):
        c = x / (numSegments - 1)
        segment = [c,c,c]  
        segments.append(segment)
    return segments

In [6]:
def getUniformColorPattern(numSegments):
    segments = []
    for r in range(numSegments):
        for g in range(numSegments):
            for b in range(numSegments):
                cR = r / (numSegments - 1)
                cG = g / (numSegments - 1)
                cB = b / (numSegments - 1)
                segment = [cR,cG,cB]  
                segments.append(segment)
    return segments

In [7]:
import math

def displayPalette(palette):
    length = len(palette)
    
    maxHeight = 1
    for x in range(2,math.floor(length**0.5)):
        if length % x == 0:
            maxHeight = x
            
    width, height = (length//maxHeight,maxHeight)
    img = numpy.zeros((height,width,3))
    
    i = 0
    
    for y in range(width):
        for x in range(height):
            img[x,y] = palette[i]
            i += 1
            
    io.imshow(img)
    io.show()

In [20]:
def update2(image, numSegments, paletteOption):
    im = io.imread(image)
    
    im = util.img_as_float64(im)
    im = im[:,:,:3]
    
    if paletteOption == "K-Means":
        palette = getKmeansPalette(im,numSegments, 0.001)
    elif paletteOption == "Greyscale":
        palette = getGrayscalePalette(numSegments)
    elif paletteOption == "Uniform":
        numSegments = min(numSegments,6)
        palette = getUniformColorPattern(numSegments)
    
    displayPalette(palette)    
      
    io.imshow(im)
    io.show()
    
    segmented = segment(im,palette)
    io.imshow(segmented)
    io.show()
    
    segmentedDithering = segmentDithering(im,palette)
    io.imshow(segmentedDithering)
    io.show()
    
    #io.imsave("dith.png",segmentedDithering)
    

numSegs = widgets.IntSlider(value = 2,min = 2,max = 32,description='Number of segments: ',continuous_update = False)

paletteOption = widgets.Dropdown(
    options=["Greyscale","Uniform","K-Means"],
    description='Palette: ',
    disabled=False,
)

interactive(update2, image=inputImage, numSegments=numSegs, paletteOption=paletteOption) 

#     try:
#         segmented = io.imread("Generated/" + image.split(".")[0] + str(numSegments) + ".png")
#     except:
#         print("Image not generated, calculating new")

interactive(children=(Dropdown(description='Bilde: ', options={'Strand': 'strand.jpeg', 'Mandrill': 'mandrill.…